# Satellite Entry/Exit Event Detection with Ground Sensor Field of View (Circular FoV)

The FoV of the ground sensor is represented as a **Circular** shape. The sensor's attitude can be adjusted to deviate from the zenith orientation by specifying three consecutive rotation angles. If no specific attitude adjustment is needed, these rotation angles should be set to 0 degrees.

Additionally, the code includes three detectors:
- GroundFieldOfViewDetector
- ElevationDetector
- BooleanDetector


In [3]:
import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

from org.orekit.attitudes import NadirPointing, LofOffset
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.frames import FramesFactory, LOFType, TopocentricFrame, Frame, Transform
from org.orekit.time import TimeScalesFactory, AbsoluteDate
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.models.earth.tessellation import EllipsoidTessellator
from org.orekit.propagation.events import EventDetector, EventsLogger, ElevationDetector, GroundFieldOfViewDetector, BooleanDetector
from org.orekit.propagation.events.handlers import ContinueOnEvent, StopOnEvent
from org.orekit.geometry.fov import CircularFieldOfView

from org.hipparchus.geometry.euclidean.threed import Vector3D, Line, RotationOrder, Rotation, RotationConvention
from org.hipparchus.geometry.spherical.twod import SphericalPolygonsSet

from math import radians, degrees, pi, sqrt, atan
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

First call the frames and time scales to be used in the code and create Earth body.

In [5]:
UTC = TimeScalesFactory.getUTC()                               # Define UTC time scale.
ECI = FramesFactory.getEME2000()                               # Define ECI reference frame.
ECEF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF reference frame.
TEME = FramesFactory.getTEME()                                 # Define TEME reference frame. 
ITRF = ECEF

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.

Here, we first take TLE as an input, and from the *TLEPropagator* method we recieve its PV to get Kepler Elements.

In [7]:
# ISS 25544
tle_line1 = "1 25544U 98067A   24265.15389185  .00020992  00000-0  38244-3 0  9995"
tle_line2 = "2 25544  51.6375 200.0394 0007352  15.0244 345.0962 15.49404240473433"
mytle = TLE(tle_line1, tle_line2)


initialDate = mytle.getDate()                    # This is the TLE epoch date read from first line.
finalDate = initialDate.shiftedBy(3600.0 * 72)   # Shift initial date by x hours.

print('Propagation initial date: ', initialDate)
print('Propagation final date: ', finalDate)

satellite_mass = 466615.0                        # Satellite mass [kg]
attitudeProvider = NadirPointing(TEME, earth)    # Also provide attitude of the satellite

SGP4 = TLEPropagator.selectExtrapolator(mytle, attitudeProvider, satellite_mass) 

Propagation initial date:  2024-09-21T03:41:36.25584Z
Propagation final date:  2024-09-24T03:41:36.25584Z


Geodetic coordinates (lat/long/alt) of the ground station must be defined below. Coordinate frame of the station is accepted as *TopocentricFrame*. This frame is associated to a position near the surface of a body shape.

The origin of the frame is at the defined geodetic point location, and the right-handed canonical trihedra is:

- X axis in the local horizontal plane (normal to zenith direction) and following the local parallel towards East
- Y axis in the horizontal plane (normal to zenith direction) and following the local meridian towards North
- Z axis towards Zenith direction

In [9]:
# Define the ground station location
latitude  = input('Latitude (degree):  ') 
latitude  = radians(float(latitude))
longitude = input('Longitude (degree):  ')
longitude = radians(float(longitude))
altitude  = input('Altitude (meter):  ')
altitude  = (float(altitude))

# Frame of the ground station is given as Topocentric Frame.
station_gp = GeodeticPoint(latitude, longitude, altitude)
station_frame = TopocentricFrame(earth, station_gp, "topo")

Latitude (degree):   32.32
Longitude (degree):   39.39
Altitude (meter):   0


In the following parts, we are going to create ground sensor with different FoV shapes using related lat/long/alt coordinates.

### Rotations with Respective Axes

The ground station frame is defined as topocentric. Based on this definition, the user can rotate this frame to achieve the desired attitude for the ground sensor.

In this code, **rotations are applied in the X-Y-Z order**. This means the topocentric frame is first rotated around the X axis, followed by the Y axis, and then the Z axis. While the rotation order can be modified, it's important to note that starting with a rotation around the Z axis is more prone to errors.

In [13]:
# Using MINUS for X and Y axes, and PLUS for Z axis in this code. Our notation becomes:
# (X = East, Y = North, Z = Zenith)
# which is the definition of topocentric frame in Orekit.
rotation1 = input('Enter the first rotation angle in degrees:  ')
rotation1 = radians(float(rotation1))
rotation2 = input('Enter the second rotation angle in degrees:  ')
rotation2 = radians(float(rotation2))
rotation3 = input('Enter the third rotation angle in degrees:  ')
rotation3 = radians(float(rotation3))

# If rotation vectors are entered PLUS instead of MINUS, then the correct XYZ notation becomes:
# (X = West, Y = South, Z = Zenith)

# First rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_I, rotation1, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(station_frame, rotation, "myFovFrame")
# Second rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_J, rotation2, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")
# Third rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.PLUS_K, rotation3, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")

Enter the first rotation angle in degrees:   0
Enter the second rotation angle in degrees:   0
Enter the third rotation angle in degrees:   0


After rotating our initial topocentric frame three times, we reach our desired attitude for the sensor attached to the ground station.

### Creating the Ground FoV Sensor 

Defining FoV shape of the ground sensor and creating *GroundFieldOfViewDetector* together with *ElevationDetector* method. Finally these event detectors are combined with a *BooleanDetector*, and they are added to propagator for monitoring.

In [17]:
# Circular Field-Of-View
#xfov  = 20.0              # [degrees]   
xfov  = input('The half-angle of the apex of the cone formed by the target (cone base) and the ground circular detector (cone apex) (degrees):  ')
xfov  = float(xfov)
fov_margin = 0.0

imagerFOV = CircularFieldOfView(Vector3D.PLUS_K, radians(xfov), fov_margin)

# Define Ground Field of View Detector
GFoVDetector = GroundFieldOfViewDetector(myFoVFrame, imagerFOV).withMaxCheck(5.0).withHandler(ContinueOnEvent())
GFoV_logger = EventsLogger()

# Define Elevation Detector
horizon_limit = radians(0.0)
eleDetector = ElevationDetector(station_frame).withConstantElevation(horizon_limit).withHandler(ContinueOnEvent())
ele_logger = EventsLogger()

#combinedDetector = BooleanDetector.andCombine([BooleanDetector.notCombine(GFoVDetector),eleDetector])
combinedDetector = BooleanDetector.andCombine([BooleanDetector.notCombine(GFoVDetector),eleDetector])
combined_logger = EventsLogger()

# Add the seperate detectors to the SGP4 propagator
SGP4.addEventDetector(GFoV_logger.monitorDetector(GFoVDetector))
SGP4.addEventDetector(ele_logger.monitorDetector(eleDetector))
SGP4.addEventDetector(combined_logger.monitorDetector(combinedDetector))

The half-angle of the apex of the cone formed by the target (cone base) and the ground circular detector (cone apex) (degrees):   25


Now start propagating the TLE with SGP4. Here, propagation interval has to be sufficiently small because if it is large, detectors may miss certain events.

In [19]:
pos = []                                                       # position vector array to be filled.
SGP4 = PVCoordinatesProvider.cast_(SGP4)
###Start SGP4 propagation from initialDate up until finalDate
while (initialDate.compareTo(finalDate) <= 0.0):
    SGP4_pv = SGP4.getPVCoordinates(initialDate, ITRF)         # Get PV coordinates
    posSGP4 = SGP4_pv.getPosition()                            # But we only want position vector.
    pos.append((posSGP4.getX(),posSGP4.getY(),posSGP4.getZ())) # Get individual elements of position
    posSGP4 = pos
    initialDate = initialDate.shiftedBy(10.0)                  # Propagate with 10 sec intervals.

Finally the logged events are retrieved and printed in a readable way.

**Ground Field of View Detector** 

In [22]:
gfov_events = GFoV_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in gfov_events: 
    if not event.isIncreasing():
        gfov_entry = event.getState().getDate()
        entry_str = str(gfov_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        gfov_exit = event.getState().getDate()
        exit_str = str(gfov_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        gfov_duration = gfov_exit.durationFrom(gfov_entry) / 60  # Convert to minutes
        duration_list.append(gfov_duration)

# Create the DataFrame
df_gfov = pd.DataFrame({
    'GFoV Entry': entry_list,
    'GFoV Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_gfov)


,GFoV Entry,GFoV Exit,Duration (min)
0,2024-09-23 ==> 11:35:30.02782853973246Z,2024-09-23 ==> 11:36:01.4030665980785Z,0.522921
1,2024-09-23 ==> 19:44:43.59295891017643Z,2024-09-23 ==> 19:45:23.87070628906552Z,0.671296


**Elevation Detector** 

In [24]:
ele_events = ele_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in ele_events: 
    if event.isIncreasing():
        ele_entry = event.getState().getDate()
        entry_str = str(ele_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        ele_exit = event.getState().getDate()
        exit_str = str(ele_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        ele_duration = ele_exit.durationFrom(ele_entry) / 60  # Convert to minutes
        duration_list.append(ele_duration)

# Create the DataFrame
df_ele = pd.DataFrame({
    'Elevation Entry': entry_list,
    'Elevation Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_ele)

,Elevation Entry,Elevation Exit,Duration (min)
0,2024-09-21 ==> 11:30:06.5968397148828Z,2024-09-21 ==> 11:39:49.07620138853702Z,9.707989
1,2024-09-21 ==> 13:06:19.63809645173625Z,2024-09-21 ==> 13:16:47.68992643520878Z,10.467530
2,2024-09-21 ==> 14:45:40.9846746831693Z,2024-09-21 ==> 14:52:44.49653641284313Z,7.058531
3,2024-09-21 ==> 16:26:09.64151862541522Z,2024-09-21 ==> 16:29:25.33492112747032Z,3.261557
4,2024-09-21 ==> 18:02:48.704279659255Z,2024-09-21 ==> 18:09:56.11815514665969Z,7.123565
5,2024-09-21 ==> 19:38:45.11280464009924Z,2024-09-21 ==> 19:49:18.5129401698631Z,10.556669
6,2024-09-21 ==> 21:15:42.71999607193062Z,2024-09-21 ==> 21:25:33.0483952283447Z,9.838807
7,2024-09-22 ==> 10:43:08.04729354293681Z,2024-09-22 ==> 10:51:26.8689359597056Z,8.313694
8,2024-09-22 ==> 12:18:16.52083483240853Z,2024-09-22 ==> 12:29:02.54716353705666Z,10.767105
9,2024-09-22 ==> 13:56:56.02055649876915Z,2024-09-22 ==> 14:05:07.05650597276352Z,8.183932


**Satellite Entry_Exit with Ground Sensor Field of View** 

In [26]:
comb_events = combined_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in comb_events: 
    if event.isIncreasing():
        comb_entry = event.getState().getDate()
        entry_str = str(comb_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        comb_exit = event.getState().getDate()
        exit_str = str(comb_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        comb_duration = comb_exit.durationFrom(comb_entry) / 60  # Convert to minutes
        duration_list.append(gfov_duration)

# Create the DataFrame
df_comb = pd.DataFrame({
    'Combined Entry': entry_list,
    'Combined Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_comb)

,Combined Entry,Combined Exit,Duration (min)
0,2024-09-23 ==> 11:35:30.02782853973246Z,2024-09-23 ==> 11:36:01.4030665980785Z,0.671296
1,2024-09-23 ==> 19:44:43.59295891017643Z,2024-09-23 ==> 19:45:23.87070628906552Z,0.671296
